# Importações

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict

pd.set_option('display.max_columns', 100)

In [2]:
anos = [2014,2015,2016,2017,2018]
numeroDeRodadas = 38
profundidadeDoIndice = 5
posicoesAtaque = ['ata','mei']
posicoesDefesa = ['lat','zag','gol']
pontuacaoUtilizada = 'atletas_media_num'

# Leitura dos arquivos

### Arquivo de times

In [3]:
times = pd.read_csv('../data/times_ids.csv')

dict_times_nomeCartola = dict(zip(times['nome.cbf'], times['nome.cartola']))
dict_times_nomeCartola['Ceará - CE']='Ceará'

dict_cod_times_nomeCartola = dict(zip(times['cod.older'].astype(np.float64), times['nome.cartola']))

### Arquivo de posições

In [4]:
posicoes = pd.read_csv('../data/posicoes_ids.csv')

dict_posicoes = dict(zip(posicoes['Cod'].astype(np.float64), posicoes['abbr']))

### Tratando arquivos de partidas

In [5]:
partidas = pd.DataFrame()
for ano in anos:    
    nomeDoArquivoPartidas= '../data/'+ str(ano) + '/' + str(ano) + '_partidas.csv'
    partidasAno = pd.read_csv(nomeDoArquivoPartidas)    
    partidasAno['home_score'] = ''
    partidasAno['away_score'] = ''
    for index, partida in partidasAno.iterrows():
        resultados = partida['score'].split(" x ")
        if (len(resultados)==2):
            partida['home_score'] = resultados[0]
            partida['away_score'] = resultados[1]
        partidasAno.iloc[partida.name] = partida
        
    partidasAno['Ano'] = [ano for i in range(0,len(partidasAno))]
    partidasAno = partidasAno.query('away_score != "" and home_score != ""')
    partidas = partidas.append(partidasAno,sort=False)  

partidas['home_score'] = partidas['home_score'].astype(np.float64)  
partidas['away_score'] = partidas['away_score'].astype(np.float64) 
partidas['home_team'] = partidas['home_team'].astype(np.str).map(dict_times_nomeCartola)
partidas['away_team'] = partidas['away_team'].astype(np.str).map(dict_times_nomeCartola) 
partidas['Ano'] = partidas['Ano'].astype(np.int32) 
partidas = partidas.reset_index()

In [6]:
partidas.shape

(1899, 13)

In [7]:
partidas.head()

,index,Unnamed: 0,game,round,date,home_team,score,away_team,arena,X,home_score,away_score,Ano
0,0,1.0,1,1,20/04/2014 - 18:30,Flamengo,0 x 0,Goiás,Mané Garrincha - Brasilia - DF,NaN,0.0,0.0,2014
1,1,2.0,2,1,19/04/2014 - 18:30,Fluminense,3 x 0,Figueirense,Maracanã - Rio de Janeiro - RJ,NaN,3.0,0.0,2014
2,2,3.0,3,1,20/04/2014 - 16:00,São Paulo,3 x 0,Botafogo,Morumbi - Sao Paulo - SP,NaN,3.0,0.0,2014
3,3,4.0,4,1,20/04/2014 - 18:30,Santos,1 x 1,Sport,Vila Belmiro - Santos - SP,NaN,1.0,1.0,2014
4,4,5.0,5,1,20/04/2014 - 16:00,Atlético-PR,1 x 0,Grêmio,Orlando Scarpelli - Florianopolis - SC,NaN,1.0,0.0,2014


### Tratando arquivos de scoults

In [8]:
scouts = pd.DataFrame()

for ano in anos:    
    inicioDosAquivos='../data/' + str(ano)
    if (ano == 2018):
        for numeroDaRodada in range(1, numeroDeRodadas+1):
            nomeDoArquivoScoutsCsv= inicioDosAquivos + '/rodada-' + str(numeroDaRodada) + '.csv'
            scout = pd.read_csv(nomeDoArquivoScoutsCsv)
            scout['Ano'] = [ano for i in range(0,len(scout))]
            scouts = scouts.append(scout,sort=False)
    else:
        nomeDoArquivoScoutsCsv= inicioDosAquivos + '/'+ str(ano) + '_scouts_raw.csv'
        scout = pd.read_csv(nomeDoArquivoScoutsCsv)
        scout['Ano'] = [ano for i in range(0,len(scout))]  
        if (ano!=2017):
            nomeDoArquivoJogadoresCsv= inicioDosAquivos +  '/'+ str(ano) + '_jogadores.csv'
            jogadores = pd.read_csv(nomeDoArquivoJogadoresCsv)
            jogadores['PosicaoID'] = jogadores['PosicaoID'].map(dict_posicoes)
            dict_idJogadores_posicoes = dict(zip(jogadores['ID'].astype(np.float64), jogadores['PosicaoID']))
            scout['atletas.posicao_id'] = scout['AtletaID'].map(dict_idJogadores_posicoes)
            
            scout = scout.query('ClubeID == ClubeID')
            scout['atletas.clube.id.full.name'] = scout['ClubeID'].map(dict_cod_times_nomeCartola) 
            scout['atletas.clube.id.full.name'] = scout['ClubeID'].map(dict_cod_times_nomeCartola) 
            scout['atletas.pontos_num'] = scout['Pontos'].astype(np.float64)  
            scout['atletas.media_num'] = scout['PontosMedia'].astype(np.float64)
            scout['atletas.preco_num'] = scout['Preco'].astype(np.float64)  
            scout['atletas.variacao_num'] = scout['PrecoVariacao'].astype(np.float64)
            scout['atletas.variacao_num'] = scout['PrecoVariacao'].astype(np.float64)
        scouts = scouts.append(scout,sort=False)
scouts.columns = scouts.columns.map(lambda x : x.replace('.','_') if isinstance(x, (str)) else x)
scouts = scouts.query('atletas_clube_id_full_name == atletas_clube_id_full_name and atletas_rodada_id!=0')

In [9]:
scouts.shape

(139728, 52)

In [10]:
scouts.head()

,AtletaID,atletas_rodada_id,ClubeID,Participou,Posicao,Jogos,Pontos,PontosMedia,Preco,PrecoVariacao,Partida,Mando,Titular,Substituido,TempoJogado,Nota,FS,PE,A,FT,FD,FF,G,I,PP,RB,FC,GC,CA,CV,SG,DD,DP,GS,Ano,atletas_posicao_id,atletas_clube_id_full_name,atletas_pontos_num,atletas_media_num,atletas_preco_num,atletas_variacao_num,Unnamed: 0,athletes_atletas_scout,atletas_apelido,atletas_atleta_id,atletas_clube_id,atletas_foto,atletas_jogos_num,atletas_nome,Rodada,atletas_status_id,atletas_slug
1,36443.0,1.0,285.0,1.0,1.0,1.0,5.0,5.00,10.60,1.60,179879.0,1.0,1.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2014,gol,Internacional,5.0,5.00,10.60,1.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36443.0,2.0,285.0,1.0,1.0,2.0,-3.0,1.00,8.27,-2.33,179882.0,0.0,1.0,0.0,1.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2014,gol,Internacional,-3.0,1.00,8.27,-2.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,36443.0,3.0,285.0,1.0,1.0,3.0,-2.6,-0.20,6.81,-1.46,179904.0,1.0,1.0,0.0,1.0,6.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2014,gol,Internacional,-2.6,-0.20,6.81,-1.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,36443.0,4.0,285.0,1.0,1.0,4.0,4.0,0.85,7.96,1.15,179922.0,1.0,1.0,0.0,1.0,6.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,2014,gol,Internacional,4.0,0.85,7.96,1.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,36443.0,5.0,285.0,1.0,1.0,5.0,5.0,1.68,8.61,0.65,179943.0,0.0,1.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2014,gol,Internacional,5.0,1.68,8.61,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Inserções dos dados analisados

### Valores do time mandante

In [12]:
def HomeTeamValues():
    partidas['AttackStrengthHomeTeam_jogo']=''
    partidas['DefenseStrengthHomeTeam_jogo']=''
    partidas['StrengthHomeTeam_jogo']=''
    partidas['AttackValueHomeTeam_jogo']=''
    partidas['DefenseValueHomeTeam_jogo']=''
    partidas['ValueHomeTeam_jogo']=''
    for time_id in dict_times_nomeCartola:
        partidasTimeDaCasa = partidas.query('home_team == "'+dict_times_nomeCartola[time_id]+'"')
        for index, partidaTimeDaCasa in partidasTimeDaCasa.iterrows():
            scoutsDaPartida = scouts.query('atletas_rodada_id == '+str(partidaTimeDaCasa['round']) +' and Ano == '+str(partidaTimeDaCasa['Ano']) + ' and atletas_clube_id_full_name == "'+ partidaTimeDaCasa['home_team'] +'"')
            scoutsDeAtaque = scoutsDaPartida[scoutsDaPartida['atletas_posicao_id'].isin(posicoesAtaque)]
            scoutsDeDefesa = scoutsDaPartida[scoutsDaPartida['atletas_posicao_id'].isin(posicoesDefesa)]
            
            partidaTimeDaCasa['AttackStrengthHomeTeam_jogo']=scoutsDeAtaque['atletas_pontos_num'].mean(axis = 0)
            partidaTimeDaCasa['DefenseStrengthHomeTeam_jogo']=scoutsDeDefesa['atletas_pontos_num'].mean(axis = 0)
            partidaTimeDaCasa['StrengthHomeTeam_jogo']=scoutsDaPartida['atletas_pontos_num'].mean(axis = 0)            
            
            partidaTimeDaCasa['AttackValueHomeTeam_jogo']=scoutsDeAtaque['atletas_pontos_num'].mean(axis = 0)
            partidaTimeDaCasa['DefenseValueHomeTeam_jogo']=scoutsDeDefesa['atletas_pontos_num'].mean(axis = 0)
            partidaTimeDaCasa['ValueHomeTeam_jogo']=scoutsDaPartida['atletas_pontos_num'].mean(axis = 0)   
            
            partidas.iloc[partidaTimeDaCasa.name] = partidaTimeDaCasa  
HomeTeamValues()    

### Valores do time visitante

In [13]:
def AwayTeamValues():
    partidas['AttackStrengthAwayTeam_jogo']=''
    partidas['DefenseStrengthAwayTeam_jogo']=''
    partidas['StrengthAwayTeam_jogo']=''
    partidas['AttackValueAwayTeam_jogo']=''
    partidas['DefenseValueAwayTeam_jogo']=''
    partidas['ValueAwayTeam_jogo']=''
    for time_id in dict_times_nomeCartola:
        partidasTimeVisitante = partidas.query('away_team == "'+dict_times_nomeCartola[time_id]+'"')
        for index, partidaTimeVisitante in partidasTimeVisitante.iterrows():
            scoutsDaPartida = scouts.query('atletas_rodada_id == '+str(partidaTimeVisitante['round']) +' and Ano == '+str(partidaTimeVisitante['Ano']) + ' and atletas_clube_id_full_name == "'+ partidaTimeVisitante['away_team'] +'"')
            scoutsDeAtaque = scoutsDaPartida[scoutsDaPartida['atletas_posicao_id'].isin(posicoesAtaque)]
            scoutsDeDefesa = scoutsDaPartida[scoutsDaPartida['atletas_posicao_id'].isin(posicoesDefesa)]
            partidaTimeVisitante['AttackStrengthAwayTeam_jogo']=scoutsDeAtaque['atletas_pontos_num'].mean(axis = 0)
            partidaTimeVisitante['DefenseStrengthAwayTeam_jogo']=scoutsDeDefesa['atletas_pontos_num'].mean(axis = 0)
            partidaTimeVisitante['StrengthAwayTeam_jogo']=scoutsDaPartida['atletas_pontos_num'].mean(axis = 0)            
            
            partidaTimeVisitante['AttackValueAwayTeam_jogo']=scoutsDeAtaque['atletas_pontos_num'].mean(axis = 0)
            partidaTimeVisitante['DefenseValueAwayTeam_jogo']=scoutsDeDefesa['atletas_pontos_num'].mean(axis = 0)
            partidaTimeVisitante['ValueAwayTeam_jogo']=scoutsDaPartida['atletas_pontos_num'].mean(axis = 0)
            partidas.iloc[partidaTimeVisitante.name] = partidaTimeVisitante  
AwayTeamValues()    

### Seta resultado da partida

In [14]:
def setaResultadoDoJogo(partida):
    if (partida['home_score']==partida['away_score']):
        return 0
    elif (partida['home_score'] > partida['away_score']):
        return 1
    else:
        return -1
partidas["resultado"] = partidas.apply(lambda partida: setaResultadoDoJogo(partida), axis=1)

### Média de gols no torneio e Índices nos últimos jogos do torneio

In [17]:
def CalculaMediaDeDoisVetores(vetor1,vetor2):
    if (vetor1.size + vetor2.size>0):
        return (vetor1.mean(axis = 0)*vetor1.size + vetor2.mean(axis = 0)*vetor2.size)/(vetor1.size + vetor2.size)
    else:
        return np.nan
    
def CalculaMediasDeGolsUltimosJogos(ultimaPartidasComoMandante,ultimaPartidasComoVisitante):    
    mediaDeGolsAFavor = CalculaMediaDeDoisVetores(ultimaPartidasComoMandante['home_score'],ultimaPartidasComoVisitante['away_score']) 
    mediaDeGolsContra = CalculaMediaDeDoisVetores(ultimaPartidasComoMandante['away_score'],ultimaPartidasComoVisitante['home_score'])
    
    return mediaDeGolsAFavor, mediaDeGolsContra

def CalculaForcasUltimosJogos(ultimaPartidasComoMandante,ultimaPartidasComoVisitante):    
    mediaForcaAtaque = CalculaMediaDeDoisVetores(ultimaPartidasComoMandante['AttackStrengthHomeTeam_jogo'],ultimaPartidasComoVisitante['AttackStrengthAwayTeam_jogo']) 
    mediaForcaDefesa = CalculaMediaDeDoisVetores(ultimaPartidasComoMandante['DefenseStrengthHomeTeam_jogo'],ultimaPartidasComoVisitante['DefenseStrengthAwayTeam_jogo']) 
    mediaForcaGeral = CalculaMediaDeDoisVetores(ultimaPartidasComoMandante['StrengthHomeTeam_jogo'],ultimaPartidasComoVisitante['StrengthAwayTeam_jogo']) 
    
    return mediaForcaAtaque, mediaForcaDefesa, mediaForcaGeral

def CalculaValoresUltimosJogos(ultimaPartidasComoMandante,ultimaPartidasComoVisitante):    
    mediaValorAtaque = CalculaMediaDeDoisVetores(ultimaPartidasComoMandante['AttackValueHomeTeam_jogo'],ultimaPartidasComoVisitante['AttackValueAwayTeam_jogo']) 
    mediaValorDefesa = CalculaMediaDeDoisVetores(ultimaPartidasComoMandante['DefenseValueHomeTeam_jogo'],ultimaPartidasComoVisitante['DefenseValueAwayTeam_jogo']) 
    mediaValorGeral = CalculaMediaDeDoisVetores(ultimaPartidasComoMandante['ValueHomeTeam_jogo'],ultimaPartidasComoVisitante['ValueAwayTeam_jogo']) 
    
    return mediaValorAtaque, mediaValorDefesa, mediaValorGeral

def CalculaIndiceDeResultadosUltimosJogos(ultimaPartidasComoMandante,ultimaPartidasComoVisitante):
    ultimaPartidas = ultimaPartidasComoMandante.merge(ultimaPartidasComoVisitante
    if (ultimaPartidas.shape[0]>0):
        partidasVencidas = ultimaPartidasComoMandante.query('resultado == 1').append(ultimaPartidasComoVisitante.query('resultado == -1'))
        partidasPerdidas = ultimaPartidasComoMandante.query('resultado == -1').append(ultimaPartidasComoVisitante.query('resultado == 1'))
        partidasEmpatadas = ultimaPartidas.query('resultado == 0')
        indiceVencidas = partidasVencidas.shape[0]/ultimaPartidas.shape[0]
        indicePerdidas = partidasPerdidas.shape[0]/ultimaPartidas.shape[0]
        indiceEmpatadas = partidasEmpatadas.shape[0]/ultimaPartidas.shape[0]
        return indiceVencidas, indicePerdidas, indiceEmpatadas
    else:
        return np.nan, np.nan, np.nan
    

def AverageGoalsAndIndiceResults():
    partidas['AverageGoalsInFavorHomeTeam']=''
    partidas['AverageGoalsAgainstHomeTeam']=''
    partidas['AverageGoalsInFavorAwayTeam']=''
    partidas['AverageGoalsAgainstAwayTeam']=''
    partidas['HomeWinLastFiveIndice']=''
    partidas['HomeDrawLastFiveIndice']=''
    partidas['HomeDefeatLastFiveIndice']=''
    partidas['AwayWinLastFiveIndice']=''
    partidas['AwayDrawLastFiveIndice']=''
    partidas['AwayDefeatLastFiveIndice']=''
    partidas['AttackStrengthAwayTeam']=''
    partidas['DefenseStrengthAwayTeam']=''
    partidas['StrengthAwayTeam']=''
    partidas['AttackValueAwayTeam']=''
    partidas['DefenseValueAwayTeam']=''
    partidas['ValueAwayTeam']=''
    partidas['AttackStrengthHomeTeam']=''
    partidas['DefenseStrengthHomeTeam']=''
    partidas['StrengthHomeTeam']=''
    partidas['AttackValueHomeTeam']=''
    partidas['DefenseValueHomeTeam']=''
    partidas['ValueHomeTeam']=''
    for time_id in dict_times_nomeCartola:
        partidasDoTime = partidas.query('home_team == "'+dict_times_nomeCartola[time_id]+'" or away_team == "'+dict_times_nomeCartola[time_id]+'"')
        partidasComoMandante = partidasDoTime.query('home_team == "'+dict_times_nomeCartola[time_id]+'"')
        partidasComoVisitante = partidasDoTime.query('away_team == "'+dict_times_nomeCartola[time_id]+'"')
        
        for index, partidaComoMandante in partidasComoMandante.iterrows():       
            
            ultimaPartidas = partidasDoTime.query('round < '+str(partidaComoMandante['round'])+' and Ano == '+str(partidaComoMandante['Ano'])).tail(profundidadeDoIndice)
            ultimaPartidasComoMandante = ultimaPartidas.query('home_team == "'+dict_times_nomeCartola[time_id]+'"')
            ultimaPartidasComoVisitante = ultimaPartidas.query('away_team == "'+dict_times_nomeCartola[time_id]+'"')
            
            mediaDeGolsAFavor, mediaDeGolsContra = CalculaMediasDeGolsUltimosJogos(ultimaPartidasComoMandante,ultimaPartidasComoVisitante)
            partidaComoMandante['AverageGoalsInFavorHomeTeam'] = mediaDeGolsAFavor
            partidaComoMandante['AverageGoalsAgainstHomeTeam'] = mediaDeGolsContra
            
            indiceVencidas, indicePerdidas, indiceEmpatadas = CalculaIndiceDeResultadosUltimosJogos(ultimaPartidasComoMandante,ultimaPartidasComoVisitante)            
            partidaComoMandante['HomeWinLastFiveIndice'] = indiceVencidas
            partidaComoMandante['HomeDrawLastFiveIndice'] = indiceEmpatadas
            partidaComoMandante['HomeDefeatLastFiveIndice'] = indicePerdidas            
            
            mediaForcaAtaque, mediaForcaDefesa, mediaForcaGeral = CalculaForcasUltimosJogos(ultimaPartidasComoMandante,ultimaPartidasComoVisitante)
            partidaComoMandante['AttackStrengthHomeTeam']=mediaForcaAtaque
            partidaComoMandante['DefenseStrengthHomeTeam']=mediaForcaDefesa
            partidaComoMandante['StrengthHomeTeam']=mediaForcaGeral
            
            mediaValorAtaque, mediaValorDefesa, mediaValorGeral = CalculaValoresUltimosJogos(ultimaPartidasComoMandante,ultimaPartidasComoVisitante)
            partidaComoMandante['AttackValueHomeTeam']=mediaValorAtaque
            partidaComoMandante['DefenseValueHomeTeam']=mediaValorDefesa
            partidaComoMandante['ValueHomeTeam']=mediaValorGeral
            
            partidas.iloc[partidaComoMandante.name] = partidaComoMandante
        
        for index, partidaComoVisitante in partidasComoVisitante.iterrows():      
            ultimaPartidas = partidasDoTime.query('round < '+str(partidaComoVisitante['round'])+' and Ano == '+str(partidaComoVisitante['Ano'])).tail(profundidadeDoIndice)
            ultimaPartidasComoMandante = ultimaPartidas.query('home_team == "'+dict_times_nomeCartola[time_id]+'"')
            ultimaPartidasComoVisitante = ultimaPartidas.query('away_team == "'+dict_times_nomeCartola[time_id]+'"')
            
            mediaDeGolsAFavor, mediaDeGolsContra = CalculaMediasDeGolsUltimosJogos(ultimaPartidasComoMandante,ultimaPartidasComoVisitante)
            partidaComoVisitante['AverageGoalsInFavorAwayTeam'] = mediaDeGolsAFavor
            partidaComoVisitante['AverageGoalsAgainstAwayTeam'] = mediaDeGolsContra
            
            indiceVencidas, indicePerdidas, indiceEmpatadas = CalculaIndiceDeResultadosUltimosJogos(ultimaPartidasComoMandante,ultimaPartidasComoVisitante)            
            partidaComoVisitante['AwayWinLastFiveIndice'] = indiceVencidas
            partidaComoVisitante['AwayDrawLastFiveIndice'] = indiceEmpatadas
            partidaComoVisitante['AwayDefeatLastFiveIndice'] = indicePerdidas
            
            mediaForcaAtaque, mediaForcaDefesa, mediaForcaGeral = CalculaForcasUltimosJogos(ultimaPartidasComoMandante,ultimaPartidasComoVisitante)
            partidaComoMandante['AttackStrengthAwayTeam']=mediaForcaAtaque
            partidaComoMandante['DefenseStrengthAwayTeam']=mediaForcaDefesa
            partidaComoMandante['StrengthAwayTeam']=mediaForcaGeral
            
            mediaValorAtaque, mediaValorDefesa, mediaValorGeral = CalculaValoresUltimosJogos(ultimaPartidasComoMandante,ultimaPartidasComoVisitante)
            partidaComoMandante['AttackValueAwayTeam']=mediaValorAtaque
            partidaComoMandante['DefenseValueAwayTeam']=mediaValorDefesa
            partidaComoMandante['ValueAwayTeam']=mediaValorGeral
            
            partidas.iloc[partidaComoVisitante.name] = partidaComoVisitante
            
AverageGoalsAndIndiceResults()

### Verifica participação de jogador

In [ ]:
scouts.head()

### Remove colunas e linhas desnecessárias

In [ ]:
partidas = partidas.drop(columns=['index','game','round','date','home_team','score','away_team','arena','home_score','away_score','Ano'])
partidas.drop(partidas.columns[partidas.columns.str.contains('X',case = False)],axis = 1, inplace = True)
partidas = partidas.loc[:, ~partidas.columns.str.contains('^Unnamed')]
partidas = partidas.dropna()

## Exportar arquivo final 

In [ ]:
partidas.head(40)

In [ ]:
partidas.tail(40)

In [ ]:
partidas.shape

In [ ]:

export_csv = partidas.to_csv ('../data/dados_limpos.csv', index = None, header=True)